In [1]:
!pip install transformers torch tqdm nltk 
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/MyDrive/text.txt', 'r') as file:
    text = file.read()



In [5]:
import nltk
nltk.download('punkt')
from sklearn.model_selection import train_test_split
# Split the text into sentences
sentences = nltk.tokenize.sent_tokenize(text)

# Split sentences into training and validation sets
train_sentences, val_sentences = train_test_split(sentences, test_size=0.2, random_state=42)

print(f"Number of training sentences: {len(train_sentences)}")
print(f"Number of validation sentences: {len(val_sentences)}")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Number of training sentences: 9968
Number of validation sentences: 2492


In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Define the model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token


In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

EPOCHS = 10  # Replace with the number of epochs you want to train for

# Define the PyTorch Dataset
class ShakespeareDataset(Dataset):
    def __init__(self, sentences, tokenizer):
        self.sentences = sentences
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        inputs = self.tokenizer(sentence, return_tensors='pt', truncation=True, padding='max_length', max_length=100)
        input_ids = inputs["input_ids"].squeeze().to(device)
        attention_mask = inputs["attention_mask"].squeeze().to(device)
        return {"input_ids": input_ids, "attention_mask": attention_mask}

# Setup GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the PyTorch DataLoaders
train_dataset = ShakespeareDataset(train_sentences, tokenizer)
val_dataset = ShakespeareDataset(val_sentences, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32)

# Define the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=EPOCHS*len(train_dataloader))

# Training loop
for epoch in range(EPOCHS):
    # Training
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", position=0, leave=True)
    for batch in progress_bar:
        optimizer.zero_grad()
        outputs = model(**batch, labels=batch["input_ids"])
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Update the progress bar
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    # Validation
    model.eval()
    total_val_loss = 0
    val_progress_bar = tqdm(val_dataloader, desc="Validation", position=0, leave=True)
    with torch.no_grad():
        for batch in val_progress_bar:
            outputs = model(**batch, labels=batch["input_ids"])
            loss = outputs.loss
            total_val_loss += loss.item()

            # Update the progress bar
            val_progress_bar.set_postfix({'validation_loss': '{:.3f}'.format(loss.item()/len(batch))})

    # Print losses
    print(f"Epoch: {epoch+1}")
    print(f"Training loss: {total_loss/len(train_dataloader)}")
    print(f"Validation loss: {total_val_loss/len(val_dataloader)}")

# Save the model
model.save_pretrained('/content/drive/MyDrive/gpt2_finetuned_shakespeare')



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Validation: 100%|██████████| 78/78 [00:05<00:00, 13.20it/s, validation_loss=0.544]


Epoch: 1
Training loss: 1.3295054842646306
Validation loss: 0.9318607235566164


Validation: 100%|██████████| 78/78 [00:05<00:00, 13.39it/s, validation_loss=0.525]


Epoch: 2
Training loss: 0.9309176233334419
Validation loss: 0.9016876900807406


Validation: 100%|██████████| 78/78 [00:05<00:00, 13.43it/s, validation_loss=0.514]


...

Epoch: 8
Training loss: 0.7558198089783008
Validation loss: 0.8830085007043985


Validation: 100%|██████████| 78/78 [00:05<00:00, 13.46it/s, validation_loss=0.513]


Epoch: 9
Training loss: 0.7398923797867237
Validation loss: 0.8874471584955851


Validation: 100%|██████████| 78/78 [00:05<00:00, 13.41it/s, validation_loss=0.514]


Epoch: 10
Training loss: 0.7278081828202957
Validation loss: 0.8907636098372631


In [26]:
# Using and testing it
prompt = "Blue ocean"  # Shakespearean style prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Move input_ids to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_ids = input_ids.to(device)

# Generate text
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=50, temperature=0.5, pad_token_id=tokenizer.eos_token_id)

# Decode token IDs back into text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)






Blue ocean, thy breath!


In [13]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input):
        embedded = self.embedding(input)
        lstm_out, _ = self.lstm(embedded)
        out = self.fc(lstm_out)
        return out


# Model parameters
VOCAB_SIZE = len(tokenizer.get_vocab())  # You need to define the tokenizer
EMBEDDING_DIM = 256
HIDDEN_DIM = 512

# Training parameters
EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 64  # You may need to adjust this

# Prepare your data loader
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Create the model, criterion, optimizer
lstm_model = LSTMModel(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(lstm_model.parameters(), lr=LEARNING_RATE)

# Start the training loop
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}")
    lstm_model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc="Training", position=0, leave=True)

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        labels = input_ids  # The labels are the input_ids itself
        optimizer.zero_grad()
        outputs = lstm_model(input_ids)
        loss = criterion(outputs.view(-1, VOCAB_SIZE), labels.view(-1))
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Training loss: {avg_train_loss}")

# Save the model
torch.save(lstm_model.state_dict(), "/content/drive/MyDrive/lstm_model.pth")


Epoch 1


Training: 100%|██████████| 156/156 [00:16<00:00,  9.62it/s, training_loss=0.224]


Training loss: 1.3835429749809778
Epoch 2


Training: 100%|██████████| 156/156 [00:15<00:00,  9.93it/s, training_loss=0.065]


Training loss: 0.25258838108334786
Epoch 3


...

Training loss: 0.002464224524700489
Epoch 9


Training: 100%|██████████| 156/156 [00:16<00:00,  9.63it/s, training_loss=0.001]


Training loss: 0.0016278049498886014
Epoch 10


Training: 100%|██████████| 156/156 [00:16<00:00,  9.64it/s, training_loss=0.001]


Training loss: 0.0011425493105959434


In [27]:
import torch.nn.functional as F

lstm_model.eval()

# Start prompt
start_prompt = "Blue ocean"
generated = start_prompt

# Tokenize the start prompt
input_ids = tokenizer.encode(start_prompt, return_tensors='pt').to(device)

# Generate three words
for _ in range(3):
    with torch.no_grad():
        outputs = lstm_model(input_ids)
        predictions = outputs[0, -1, :]

    # Apply softmax to predictions to get probabilities
    probabilities = F.softmax(predictions, dim=-1)

    # Sample from the distribution
    predicted_id = torch.multinomial(probabilities, 1)
    generated_word = tokenizer.decode([predicted_id.item()])
    generated += " " + generated_word

    # Prepare the new input
    input_ids = predicted_id.unsqueeze(0).to(device)

print(generated)





Blue ocean literally Happy Happy


In [28]:
from torch.nn import CrossEntropyLoss

def compute_perplexity(model, dataloader, device):
    loss_function = CrossEntropyLoss()
    total_loss = 0
    total_count = 0
    
    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = (b.to(device) for b in batch)
            
            outputs = model(inputs, labels=inputs)
            loss = loss_function(outputs.logits.view(-1, outputs.logits.size(-1)), labels.view(-1))

            total_loss += loss.item() * inputs.size(0)
            total_count += inputs.size(0)
    
    return torch.exp(torch.tensor(total_loss / total_count)).item()

validation_perplexity = compute_perplexity(model, validation_dataloader, device)
print(f'Validation Perplexity: {validation_perplexity}')


NameError: ignored

In [31]:
# Generating text
prompt = "Once upon a time"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

# Generate text
output = model.generate(input_ids, max_length=100, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)

# Decode token IDs back into text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Once upon a time of war,
When the foe is at his heels,
The king's power is at his beck;
And when he is not at his beck,
The king's power is at his beck.


In [30]:
from nltk.translate.bleu_score import sentence_bleu

def compute_bleu(original_text, generated_text):
    reference = original_text.split()
    candidate = generated_text.split()
    score = sentence_bleu([reference], candidate)
    return score

# Assuming original_text and generated_text are defined
bleu_score = compute_bleu(original_text, generated_text)
print(f'BLEU Score: {bleu_score}')


NameError: ignored